# Import Libraries

In [114]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
import pickle

In [115]:
import nltk

import re 

from nltk.stem import PorterStemmer # for stemming

from nltk.stem import WordNetLemmatizer # for lemmatization

from nltk.corpus import stopwords
nltk.download('punkt')

nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import GaussianNB

from xgboost import XGBClassifier

from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\92303\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\92303\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\92303\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# LOAD THE DATASET

In [116]:
data = pd.read_csv("tweets.csv")

In [117]:
data

,username,tweet,date,location,search
0,Kareema_H,"Im back to drinking a gallon of water a day, ...",2021-09-09 23:59:53,World,Water
1,BillsBravesFan,"Tampa man. What a great city. Weather, water, ...",2021-09-09 23:59:49,"Red Bank, SC - US",Water
2,hydratebot,my friend! i command you to consume some water...,2021-09-09 23:59:43,World,Water
3,lorena_ca21,My head's under water\nBut I'm breathing fine\...,2021-09-09 23:59:37,World,Water
4,_Ethan_Johnson_,I believe in refrigerator door water supremacy.,2021-09-09 23:59:34,World,Water
...,...,...,...,...,...
34548,burrow_JohnR,Dr. Pepper vs Coca-Cola - I can’t decide which...,2021-09-09 16:02:20,World,cola
34549,Akinbrands,The simpler the concept of a brand\nThe easier...,2021-09-09 16:02:10,World,cola
34550,finalpambot,UNDRINKABLE NUKA COLA\nMY SISYPHEAN TORTURE,2021-09-09 15:56:01,World,cola
34551,banriion,"Me, loudly; peaches you want some cokey cola?\...",2021-09-09 15:52:56,World,cola


# PRE-PROCESSING THE DATASET

In [118]:
data.shape

(34553, 5)

In [119]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34553 entries, 0 to 34552
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   username  34553 non-null  object
 1   tweet     34553 non-null  object
 2   date      34553 non-null  object
 3   location  34553 non-null  object
 4   search    34553 non-null  object
dtypes: object(5)
memory usage: 1.3+ MB


In [120]:
data.isnull().sum()

username    0
tweet       0
date        0
location    0
search      0
dtype: int64

# Choosing only the required columns

In [121]:
df = data[['tweet']]

In [122]:
df

,tweet
0,"Im back to drinking a gallon of water a day, ..."
1,"Tampa man. What a great city. Weather, water, ..."
2,my friend! i command you to consume some water...
3,My head's under water\nBut I'm breathing fine\...
4,I believe in refrigerator door water supremacy.
...,...
34548,Dr. Pepper vs Coca-Cola - I can’t decide which...
34549,The simpler the concept of a brand\nThe easier...
34550,UNDRINKABLE NUKA COLA\nMY SISYPHEAN TORTURE
34551,"Me, loudly; peaches you want some cokey cola?\..."


# Checking the Null Values

In [123]:
df.isnull().sum()

tweet    0
dtype: int64

# Removing the NULL VALUES

In [124]:
df.dropna(axis = 0, inplace = True)

In [125]:
df.isnull().sum()

tweet    0
dtype: int64

In [126]:
df.shape

(34553, 1)

In [127]:
df['tweet'].value_counts()

i need a sugar daddy                                                                                                                                                                                                                                                                                     67
I need a sugar daddy                                                                                                                                                                                                                                                                                     23
9/10/21 is tomorrow. Be ready.                                                                                                                                                                                                                                                                            8
Flan_Water TTT                                                                                      

In [128]:
df.dtypes

tweet    object
dtype: object

# Cleaning the Description Column removing punctuation marks 

In [129]:
def clean(review):

    descrip = re.sub('[^a-zA-Z]', ' ', review)

    review = review.lower()

    return review

df['tweet'] = pd.DataFrame(df['tweet'].apply(lambda x: clean(x)))

df.head()


,tweet
0,"im back to drinking a gallon of water a day, ..."
1,"tampa man. what a great city. weather, water, ..."
2,my friend! i command you to consume some water...
3,my head's under water\nbut i'm breathing fine\...
4,i believe in refrigerator door water supremacy.


In [130]:
df['tweet'].replace('[@+]', "", regex=True,inplace=True)

df['tweet'].replace('[()]', "", regex=True,inplace=True)


df['tweet'].replace('[#+]', "", regex=True, inplace = True)


#url_regex = "(https?://)(s)*(www.)?(s)*((w|s)+.)*([w-s]+/)*([w-]+)((?)?[ws]*=s*[w%&]*)*"
#df['descrip_Cleaned'] = df['descrip_Cleaned'].replace(url_regex, "", regex=True)

In [131]:
#url_regex = "(https?://)(s)*(www.)?(s)*((w|s)+.)*([w-s]+/)*([w-]+)((?)?[ws]*=s*[w%&]*)*"
#df['descrip_Cleaned'] = df['descrip_Cleaned'].replace(url_regex, "", regex=True)

In [132]:
df['tweet'] = df['tweet'].replace(to_replace=r'^https?:\/\/.*[\r\n]*',value='',regex=True)

In [133]:
df['tweet'] = df['tweet'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])


In [134]:
df

,tweet
0,"im back to drinking a gallon of water a day, ..."
1,"tampa man. what a great city. weather, water, ..."
2,my friend! i command you to consume some water...
3,my head's under water\nbut i'm breathing fine\...
4,i believe in refrigerator door water supremacy.
...,...
34548,dr. pepper vs coca-cola - i can’t decide which...
34549,the simpler the concept of a brand\nthe easier...
34550,undrinkable nuka cola\nmy sisyphean torture
34551,"me, loudly; peaches you want some cokey cola?\..."


In [135]:
url_regex = "(https?://)" 
df['tweet'] = df['tweet'].replace(url_regex, "", regex=True)

In [136]:
df

,tweet
0,"im back to drinking a gallon of water a day, ..."
1,"tampa man. what a great city. weather, water, ..."
2,my friend! i command you to consume some water...
3,my head's under water\nbut i'm breathing fine\...
4,i believe in refrigerator door water supremacy.
...,...
34548,dr. pepper vs coca-cola - i can’t decide which...
34549,the simpler the concept of a brand\nthe easier...
34550,undrinkable nuka cola\nmy sisyphean torture
34551,"me, loudly; peaches you want some cokey cola?\..."


# REMOVING STOP WORDS

In [137]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [138]:
df['tweet'] = df['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


In [139]:
df

,tweet
0,"im back drinking gallon water day, i’m tired g..."
1,"tampa man. great city. weather, water, great d..."
2,friend! command consume water. thank physical ...
3,head's water i'm breathing fine crazy i'm mind...
4,believe refrigerator door water supremacy.
...,...
34548,dr. pepper vs coca-cola - can’t decide prefer....
34549,"simpler concept brand easier remember, underst..."
34550,undrinkable nuka cola sisyphean torture
34551,"me, loudly; peaches want cokey cola? mal’s pit..."


In [140]:
df.isnull().sum()

tweet    0
dtype: int64

In [141]:
df.dropna(axis = 0, inplace = True)

In [142]:
df.isnull().sum()

tweet    0
dtype: int64

In [143]:
df['tweet'] = df['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


In [144]:
df[['tweet']]

,tweet
0,"im back drinking gallon water day, i’m tired g..."
1,"tampa man. great city. weather, water, great d..."
2,friend! command consume water. thank physical ...
3,head's water i'm breathing fine crazy i'm mind...
4,believe refrigerator door water supremacy.
...,...
34548,dr. pepper vs coca-cola - can’t decide prefer....
34549,"simpler concept brand easier remember, underst..."
34550,undrinkable nuka cola sisyphean torture
34551,"me, loudly; peaches want cokey cola? mal’s pit..."


# Vectorization of texts in ‘descrip_Cleaned’ and ‘text_Cleaned’ columns.

In [145]:
x= df['tweet']


In [146]:
#y = df['gender']
x

0        im back drinking gallon water day, i’m tired g...
1        tampa man. great city. weather, water, great d...
2        friend! command consume water. thank physical ...
3        head's water i'm breathing fine crazy i'm mind...
4               believe refrigerator door water supremacy.
                               ...                        
34548    dr. pepper vs coca-cola - can’t decide prefer....
34549    simpler concept brand easier remember, underst...
34550              undrinkable nuka cola sisyphean torture
34551    me, loudly; peaches want cokey cola? mal’s pit...
34552       every ten minutes, 6270000 coca-cola consumed.
Name: tweet, Length: 34553, dtype: object

In [147]:
cv = CountVectorizer(max_features = 2000)
x = cv.fit_transform(df['tweet']).toarray()
#x1=cv.fit_transform(df['text_Cleaned']).toarray()


In [148]:
x.shape

(34553, 2000)

In [149]:
filenameDT = 'model_pt_f.sav'

In [150]:
# load the model from disk
model = pickle.load(open(filenameDT, 'rb'))

In [151]:
y_pred = model.predict(x)

In [152]:
y_pred

array([1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0.,
       1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1.,
       1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0.,
       0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0.,
       1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1.,
       0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1.,
       0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0.,
       0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1.,
       1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
       1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1.,
       0., 0., 1., 0., 1.

In [153]:
y_pred.dtype

dtype('float64')

In [154]:
y_pred.astype("object")

array([1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0,
       1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0,
       0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0,
       0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0,
       1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0,
       1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0,
       1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0,
       0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,
       0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,
       1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0,
       1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0,
       0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0,
       1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0,
       1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.

In [155]:
y_pred = y_pred.astype(str)

In [156]:
y_pred

array(['1.0', '1.0', '1.0', '1.0', '0.0', '1.0', '1.0', '0.0', '1.0',
       '0.0', '0.0', '1.0', '0.0', '1.0', '0.0', '1.0', '0.0', '1.0',
       '1.0', '0.0', '1.0', '0.0', '0.0', '0.0', '1.0', '0.0', '0.0',
       '0.0', '1.0', '0.0', '0.0', '1.0', '0.0', '1.0', '1.0', '1.0',
       '0.0', '1.0', '0.0', '0.0', '0.0', '1.0', '0.0', '0.0', '1.0',
       '0.0', '0.0', '0.0', '0.0', '0.0', '1.0', '1.0', '1.0', '1.0',
       '1.0', '1.0', '1.0', '0.0', '1.0', '1.0', '0.0', '1.0', '1.0',
       '1.0', '0.0', '1.0', '1.0', '0.0', '0.0', '0.0', '1.0', '1.0',
       '1.0', '1.0', '0.0', '0.0', '1.0', '1.0', '1.0', '0.0', '1.0',
       '0.0', '1.0', '1.0', '0.0', '1.0', '1.0', '1.0', '1.0', '0.0',
       '1.0', '0.0', '1.0', '1.0', '0.0', '1.0', '0.0', '1.0', '0.0',
       '0.0', '0.0', '1.0', '0.0', '0.0', '0.0', '0.0', '1.0', '1.0',
       '0.0', '1.0', '0.0', '0.0', '0.0', '0.0', '0.0', '1.0', '0.0',
       '1.0', '1.0', '0.0', '0.0', '0.0', '1.0', '1.0', '1.0', '1.0',
       '1.0', '1.0',

In [157]:
y_pred[y_pred =="1.0"] = "male"
y_pred[y_pred== "0.0"] = "female"


In [158]:
import numpy
import sys
numpy.set_printoptions(threshold=sys.maxsize)


In [159]:
y_pred

array(['male', 'male', 'male', 'male', 'female', 'male', 'male', 'female',
       'male', 'female', 'female', 'male', 'female', 'male', 'female',
       'male', 'female', 'male', 'male', 'female', 'male', 'female',
       'female', 'female', 'male', 'female', 'female', 'female', 'male',
       'female', 'female', 'male', 'female', 'male', 'male', 'male',
       'female', 'male', 'female', 'female', 'female', 'male', 'female',
       'female', 'male', 'female', 'female', 'female', 'female', 'female',
       'male', 'male', 'male', 'male', 'male', 'male', 'male', 'female',
       'male', 'male', 'female', 'male', 'male', 'male', 'female', 'male',
       'male', 'female', 'female', 'female', 'male', 'male', 'male',
       'male', 'female', 'female', 'male', 'male', 'male', 'female',
       'male', 'female', 'male', 'male', 'female', 'male', 'male', 'male',
       'male', 'female', 'male', 'female', 'male', 'male', 'female',
       'male', 'female', 'male', 'female', 'female', 'female', 'm

In [160]:
y_pred = pd.DataFrame(y_pred, columns = ['gender'])

In [161]:
y_pred

,gender
0,male
1,male
2,male
3,male
4,female
...,...
34548,female
34549,female
34550,female
34551,female


In [162]:
data

,username,tweet,date,location,search
0,Kareema_H,"Im back to drinking a gallon of water a day, ...",2021-09-09 23:59:53,World,Water
1,BillsBravesFan,"Tampa man. What a great city. Weather, water, ...",2021-09-09 23:59:49,"Red Bank, SC - US",Water
2,hydratebot,my friend! i command you to consume some water...,2021-09-09 23:59:43,World,Water
3,lorena_ca21,My head's under water\nBut I'm breathing fine\...,2021-09-09 23:59:37,World,Water
4,_Ethan_Johnson_,I believe in refrigerator door water supremacy.,2021-09-09 23:59:34,World,Water
...,...,...,...,...,...
34548,burrow_JohnR,Dr. Pepper vs Coca-Cola - I can’t decide which...,2021-09-09 16:02:20,World,cola
34549,Akinbrands,The simpler the concept of a brand\nThe easier...,2021-09-09 16:02:10,World,cola
34550,finalpambot,UNDRINKABLE NUKA COLA\nMY SISYPHEAN TORTURE,2021-09-09 15:56:01,World,cola
34551,banriion,"Me, loudly; peaches you want some cokey cola?\...",2021-09-09 15:52:56,World,cola


In [163]:
df

,tweet
0,"im back drinking gallon water day, i’m tired g..."
1,"tampa man. great city. weather, water, great d..."
2,friend! command consume water. thank physical ...
3,head's water i'm breathing fine crazy i'm mind...
4,believe refrigerator door water supremacy.
...,...
34548,dr. pepper vs coca-cola - can’t decide prefer....
34549,"simpler concept brand easier remember, underst..."
34550,undrinkable nuka cola sisyphean torture
34551,"me, loudly; peaches want cokey cola? mal’s pit..."


In [164]:
data

,username,tweet,date,location,search
0,Kareema_H,"Im back to drinking a gallon of water a day, ...",2021-09-09 23:59:53,World,Water
1,BillsBravesFan,"Tampa man. What a great city. Weather, water, ...",2021-09-09 23:59:49,"Red Bank, SC - US",Water
2,hydratebot,my friend! i command you to consume some water...,2021-09-09 23:59:43,World,Water
3,lorena_ca21,My head's under water\nBut I'm breathing fine\...,2021-09-09 23:59:37,World,Water
4,_Ethan_Johnson_,I believe in refrigerator door water supremacy.,2021-09-09 23:59:34,World,Water
...,...,...,...,...,...
34548,burrow_JohnR,Dr. Pepper vs Coca-Cola - I can’t decide which...,2021-09-09 16:02:20,World,cola
34549,Akinbrands,The simpler the concept of a brand\nThe easier...,2021-09-09 16:02:10,World,cola
34550,finalpambot,UNDRINKABLE NUKA COLA\nMY SISYPHEAN TORTURE,2021-09-09 15:56:01,World,cola
34551,banriion,"Me, loudly; peaches you want some cokey cola?\...",2021-09-09 15:52:56,World,cola


In [165]:
data_labelled = pd.concat([data,y_pred],join='inner',axis=1)


In [166]:
data_labelled

,username,tweet,date,location,search,gender
0,Kareema_H,"Im back to drinking a gallon of water a day, ...",2021-09-09 23:59:53,World,Water,male
1,BillsBravesFan,"Tampa man. What a great city. Weather, water, ...",2021-09-09 23:59:49,"Red Bank, SC - US",Water,male
2,hydratebot,my friend! i command you to consume some water...,2021-09-09 23:59:43,World,Water,male
3,lorena_ca21,My head's under water\nBut I'm breathing fine\...,2021-09-09 23:59:37,World,Water,male
4,_Ethan_Johnson_,I believe in refrigerator door water supremacy.,2021-09-09 23:59:34,World,Water,female
...,...,...,...,...,...,...
34548,burrow_JohnR,Dr. Pepper vs Coca-Cola - I can’t decide which...,2021-09-09 16:02:20,World,cola,female
34549,Akinbrands,The simpler the concept of a brand\nThe easier...,2021-09-09 16:02:10,World,cola,female
34550,finalpambot,UNDRINKABLE NUKA COLA\nMY SISYPHEAN TORTURE,2021-09-09 15:56:01,World,cola,female
34551,banriion,"Me, loudly; peaches you want some cokey cola?\...",2021-09-09 15:52:56,World,cola,female


In [167]:
df = data_labelled[['tweet', 'gender']]

# Checking the Null Values

In [168]:
df.isnull().sum()

tweet     0
gender    0
dtype: int64

# Removing the NULL VALUES

In [169]:
df.dropna(axis = 0, inplace = True)

In [170]:
df.isnull().sum()

tweet     0
gender    0
dtype: int64

In [171]:
df['gender'].value_counts()

female    19561
male      14992
Name: gender, dtype: int64

# Label Encoding, Representing Male with 1 and Female with 0

In [172]:
for gen in df['gender']:

  if gen=='male':

     df['gender'].replace({'male':1},inplace=True)

  elif gen=='female':

     df['gender'].replace({'female':0},inplace=True)

df['gender'].value_counts()


0    19561
1    14992
Name: gender, dtype: int64

# Cleaning the Description Column removing punctuation marks

In [173]:
def clean(review):

    descrip = re.sub('[^a-zA-Z]', ' ', review)

    review = review.lower()

    return review

df['tweet'] = pd.DataFrame(df['tweet'].apply(lambda x: clean(x)))

df.head()


,tweet,gender
0,"im back to drinking a gallon of water a day, ...",1
1,"tampa man. what a great city. weather, water, ...",1
2,my friend! i command you to consume some water...,1
3,my head's under water\nbut i'm breathing fine\...,1
4,i believe in refrigerator door water supremacy.,0


# Cleaning the data which includes punctuation removal,number removal, and different signs like ‘@’,'()’,’#’, and URL with ‘ ‘.

In [174]:
df['tweet'].replace('[@+]', "", regex=True,inplace=True)

df['tweet'].replace('[()]', "", regex=True,inplace=True)


df['tweet'].replace('[#+]', "", regex=True, inplace = True)


In [175]:
df['tweet'] = df['tweet'].replace(to_replace=r'^https?:\/\/.*[\r\n]*',value='',regex=True)

In [176]:
df['tweet'] = df['tweet'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])


# REMOVING STOP WORDS

In [177]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [178]:
df['tweet'] = df['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


In [179]:
cv = CountVectorizer(max_features = 2000)
x = cv.fit_transform(df['tweet']).toarray()

In [180]:
x.shape

(34553, 2000)

In [181]:
y = df['gender']

# Data was split into train and test data with an 80:20 ratio.

In [182]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = 0)

In [183]:
# Vectorize text reviews to numbers
#vec = CountVectorizer(stop_words='english')
#x_train = vec.fit_transform(X_train).toarray()
#x_test = vec.transform(X_test).toarray()

In [184]:
gnbmodel = GaussianNB()
gnbmodel.fit(X_train , y_train)
y_pred = gnbmodel.predict(X_test)

In [185]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 73.20%


In [186]:
confusion_matrix(y_test, y_pred)


array([[3061,  849],
       [1003, 1998]], dtype=int64)

In [187]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.78      0.77      3910
           1       0.70      0.67      0.68      3001

    accuracy                           0.73      6911
   macro avg       0.73      0.72      0.73      6911
weighted avg       0.73      0.73      0.73      6911



# GridsearchCV() was used to tune hyperparameters for threse Classifiers.

In [188]:
param_grid_nb = {
    'var_smoothing': np.logspace(0,-9, num=100)
}

In [189]:
nbModel_grid = GridSearchCV(estimator=gnbmodel, param_grid=param_grid_nb, verbose=1, cv=3, n_jobs=-1)


In [190]:
nbModel_grid.fit(X_train, y_train)


Fitting 3 folds for each of 100 candidates, totalling 300 fits


GridSearchCV(cv=3, estimator=GaussianNB(), n_jobs=-1,
             param_grid={'var_smoothing': array([1.00000000e+00, 8.11130831e-01, 6.57933225e-01, 5.33669923e-01,
       4.32876128e-01, 3.51119173e-01, 2.84803587e-01, 2.31012970e-01,
       1.87381742e-01, 1.51991108e-01, 1.23284674e-01, 1.00000000e-01,
       8.11130831e-02, 6.57933225e-02, 5.33669923e-02, 4.32876128e-02,
       3.51119173e-02, 2.8480358...
       1.23284674e-07, 1.00000000e-07, 8.11130831e-08, 6.57933225e-08,
       5.33669923e-08, 4.32876128e-08, 3.51119173e-08, 2.84803587e-08,
       2.31012970e-08, 1.87381742e-08, 1.51991108e-08, 1.23284674e-08,
       1.00000000e-08, 8.11130831e-09, 6.57933225e-09, 5.33669923e-09,
       4.32876128e-09, 3.51119173e-09, 2.84803587e-09, 2.31012970e-09,
       1.87381742e-09, 1.51991108e-09, 1.23284674e-09, 1.00000000e-09])},
             verbose=1)

In [191]:
nbModel_grid.best_params_

{'var_smoothing': 0.003511191734215131}

In [192]:
y_pred_hyper = nbModel_grid.predict(X_test)

In [193]:
print(confusion_matrix(y_test, y_pred_hyper), ": is the confusion matrix")

[[3459  451]
 [1246 1755]] : is the confusion matrix


In [194]:
accuracy = accuracy_score(y_test, y_pred_hyper)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 75.44%


In [195]:
print(classification_report(y_test, y_pred_hyper))

              precision    recall  f1-score   support

           0       0.74      0.88      0.80      3910
           1       0.80      0.58      0.67      3001

    accuracy                           0.75      6911
   macro avg       0.77      0.73      0.74      6911
weighted avg       0.76      0.75      0.75      6911



# Lightgbm Classifier

In [196]:
lgbmodel = LGBMClassifier(max_depth=3)
lgbmodel.fit(X_train, y_train)

LGBMClassifier(max_depth=3)

In [197]:
y_pred1= lgbmodel.predict(X_test)

In [198]:
accuracy = accuracy_score(y_test, y_pred1)

print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 71.45%


# Hyper parameter tuning using Grid Search CV

In [199]:
param_grid = {
              "max_depth": [2, 3, 5, 10],
              "min_child_weight": [0.001, 0.002],
              "learning_rate": [0.05, 0.1]
              }

In [200]:
lgbgrid = GridSearchCV(estimator = lgbmodel, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 0)

lgbgrid.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=LGBMClassifier(max_depth=3), n_jobs=-1,
             param_grid={'learning_rate': [0.05, 0.1],
                         'max_depth': [2, 3, 5, 10],
                         'min_child_weight': [0.001, 0.002]})

In [201]:
lgbgrid.best_params_

{'learning_rate': 0.1, 'max_depth': 10, 'min_child_weight': 0.001}

In [202]:
y_pred1_hyper = lgbgrid.predict(X_test)

In [203]:
accuracy = accuracy_score(y_test, y_pred1_hyper)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 77.20%


In [ ]:
print(classification_report(y_test, y_pred1_hyper))

# XGBoost Classifier

In [204]:
xgbmodel = XGBClassifier(max_depth=5, min_child_weight=1)
xgbmodel.fit(X_train, y_train)

[15:55:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [205]:
y_pred2 = xgbmodel.predict(X_test)


In [206]:
accuracy = accuracy_score(y_test, y_pred2)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 79.16%


In [207]:
print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

           0       0.77      0.90      0.83      3910
           1       0.83      0.65      0.73      3001

    accuracy                           0.79      6911
   macro avg       0.80      0.78      0.78      6911
weighted avg       0.80      0.79      0.79      6911



# Hyperparameter tuning of XGBoost Classifier using GridsearchCV() function.

In [208]:
xgb_param_grid = {
              "max_depth": [3, 5],
              "min_child_weight": [1, 2],
              }

In [209]:
xgbgrid = GridSearchCV(estimator = xgbmodel, param_grid = xgb_param_grid, cv = 3, n_jobs = -1, verbose = 0)

xgbgrid.fit(X_train, y_train)

[16:00:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0, gpu_id=-1,
                                     importance_type='gain',
                                     interaction_constraints='',
                                     learning_rate=0.300000012,
                                     max_delta_step=0, max_depth=5,
                                     min_child_weight=1, missing=nan,
                                     monotone_constraints='()',
                                     n_estimators=100, n_jobs=4,
                                     num_parallel_tree=1, random_state=0,
                                     reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, subsample=1,
                                     tree_method='exact', validate_parameters=1,
         

In [210]:
xgbgrid.best_params_

{'max_depth': 3, 'min_child_weight': 1}

In [211]:
y_pred2_hyper = xgbgrid.predict(X_test)


In [212]:
y_pred2_hyper

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [213]:
accuracy = accuracy_score(y_test, y_pred2_hyper)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 76.82%


In [ ]:
print(classification_report(y_test, y_pred2_hyper))